In [1]:
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from icecream import ic
from pathlib import Path
import time
import os
import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
print("***** Put the interpreter in UTC, to make sure no TZ issues")
os.environ["TZ"] = "UTC"
time.tzset()

***** Put the interpreter in UTC, to make sure no TZ issues


In [4]:
import sys
sys.path.append("../../../..")
import ascat
from air_density import air_density

In [5]:
data_dir = "/lustre/storeB/project/IT/geout/machine-ocean/data_raw/metop/"

In [6]:
with open(data_dir + 'collocated_dict_ascat_9000_images_ifs_without_spurs1.pickle', 'rb') as handle:
        ascat_dict = pickle.load(handle)

In [7]:
ascat_dict['Endurance_4']

,CE,CN,IR,Lv,P10,Pair,Q10,Qair,RH,RH10,...,azi_angle_trip_fore,azi_angle_trip_mid,azi_angle_trip_aft,inc_angle_trip_fore,inc_angle_trip_mid,inc_angle_trip_aft,start_sensing_time,stop_sensing_time,u10,v10
datetime,,,,,,,,,,,,,,,,,,,,,
2016-10-05 04:09:59.999609,-0.017382,-0.021659,356.150002,2472.300796,1010.550340,1011.750500,8.577781,8.573971,91.529049,91.343041,...,116.559998,71.680000,26.650000,36.770000,27.480000,36.750000,20161005T033000Z,20161005T051158.125000Z,4.225077,8.839680
2016-10-05 05:09:59.995250,-0.132186,0.137583,369.484998,2472.429893,1010.575636,1011.779001,8.454130,8.456720,94.734101,94.769206,...,116.559998,71.680000,26.650000,36.770000,27.480000,36.750000,20161005T033000Z,20161005T051158.125000Z,3.755952,8.773696
2016-10-05 06:10:00.028711,-0.056678,0.245910,369.254999,2472.318806,1010.085812,1011.285999,8.972475,8.942997,96.669200,96.850041,...,-145.270004,-99.449997,-53.649998,53.599998,42.259998,53.480000,20161005T042400Z,20161005T060558.125000Z,4.952369,10.034121
2016-10-05 18:09:59.967788,0.008345,-0.011118,353.265001,2472.341633,1013.002545,1014.201999,8.437812,8.443148,85.294550,84.492424,...,56.139999,101.570000,146.879990,37.759998,28.260000,37.719997,20161005T175100Z,20161005T193258.125000Z,9.552159,-2.316962
2016-10-05 19:09:59.977450,0.003995,0.017311,356.975003,2472.281558,1013.479857,1014.679004,8.372620,8.385586,83.562800,82.509306,...,56.139999,101.570000,146.879990,37.759998,28.260000,37.719997,20161005T175100Z,20161005T193258.125000Z,9.363328,-2.101505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-20 19:09:59.961609,0.384565,0.538847,328.539998,2477.981673,980.954900,982.139502,6.157890,6.232325,86.269449,85.767931,...,52.869999,98.879997,144.729996,56.590000,45.200001,56.610001,20170120T173600Z,20170120T191758.125000Z,-3.806871,10.756151
2017-01-21 06:10:00.018663,0.301466,0.530836,325.280002,2477.817289,984.227507,985.415500,5.635275,5.740658,78.588150,77.579673,...,-143.589996,-97.419998,-51.160000,63.829998,52.379997,63.730000,20170121T043600Z,20170121T061758.125000Z,-0.949311,10.592604
2017-01-21 20:09:59.979469,0.301554,0.187796,340.339996,2476.481140,990.985359,992.178003,6.057360,6.150811,80.536600,79.690517,...,-22.949999,-67.669998,-112.449997,60.039997,48.959999,60.109997,20170121T185700Z,20170121T203858.125000Z,5.919338,7.108261


In [8]:
ifs_dir = '/lustre/storeB/project/IT/geout/machine-ocean/data_raw/ifs/other_params'

Example of file name: ifs_fc_20141101.nc

In [10]:
for buoy in ascat_dict.keys():
    print(buoy)
    if buoy != 'CLIMODE':
        df = ascat_dict[buoy]
        df['chnk'] = np.nan
        df['msl'] = np.nan
        df['t2m'] = np.nan
        df['d2m'] = np.nan
        df['zust'] = np.nan
        lat = df['lat'][0]
        lon = df['lon'][0]
        lon_360 = lon
        if lon < 0:
            lon_360 = lon + 360
        for idx in df.index:
            if (idx > datetime(2012, 1, 1,)) & (idx < datetime(2017, 1, 1,)): 
                file_date = str(idx).split(' ')[0].replace('-', '')
                ifs_fn = 'ifs_fc_' + file_date + '.nc'
                ifs_data = xr.open_dataset(ifs_dir + '/' + ifs_fn)
                ifs_data_buoy = ifs_data.sel(
                    {
                        'time' : idx, 
                        'latitude' : lat, 
                        'longitude': lon_360
                    },
                    method='nearest'
                )
                
                df['chnk'][idx] = ifs_data_buoy['chnk'].values
                df['msl'][idx] = ifs_data_buoy['msl'].values
                df['t2m'][idx] = ifs_data_buoy['t2m'].values
                df['d2m'][idx] = ifs_data_buoy['d2m'].values
                df['zust'][idx] = ifs_data_buoy['zust'].values
            #except:
            #    pass
        # Overwrite df with new columns (IFS)
        ascat_dict[buoy] = df
    


Endurance_7
Endurance_3
Pioneer_6
Endurance_8
Pioneer_8
Irminger_3
SPURS2
SouthernOcean2
Pioneer_5
Pioneer_3
Pioneer_7
Endurance_4
CLIMODE
Irminger_7
Endurance_9
Pioneer_10
Irminger_6


In [11]:
ifs_data_buoy = ifs_data.sel(
                    {
                        'time' : idx, 
                        'latitude' : lat, 
                        'longitude': lon_360
                    },
                    method='nearest'
                )

ifs_data_buoy

<xarray.Dataset>
Dimensions:    ()
Coordinates:
    longitude  float32 320.5
    latitude   float32 59.9
    time       datetime64[ns] 2016-12-31T23:00:00
Data variables:
    chnk       float32 ...
    msl        float32 ...
    t2m        float32 ...
    d2m        float32 ...
    zust       float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-08 07:39:43 GMT by grib_to_netcdf-2.26.0: grib_to_n...

In [12]:
ifs_data_buoy['chnk'].values

array(0.03801922, dtype=float32)

In [13]:
df

,CE,CN,IR,Lv,P10,Pair,Q10,Qair,RH,RH10,...,inc_angle_trip_aft,start_sensing_time,stop_sensing_time,u10,v10,chnk,msl,t2m,d2m,zust
datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-11 13:10:59.996759,0.178938,0.101390,341.660002,2474.809122,1017.426988,1018.645490,7.585522,7.604719,92.496903,92.327994,...,43.219997,20190811T123300Z,20190811T141158.125000Z,-4.992647,0.315735,NaN,NaN,NaN,NaN,NaN
2019-08-11 14:10:59.996021,0.183869,0.075291,342.504999,2474.597208,1017.185669,1018.404016,7.615780,7.633561,93.048452,92.957714,...,43.219997,20190811T123300Z,20190811T141158.125000Z,-4.700168,0.731575,NaN,NaN,NaN,NaN,NaN
2019-08-11 15:10:59.997899,0.140371,0.029420,343.119998,2474.740347,1017.022296,1018.240500,7.521966,7.544814,91.938303,91.756495,...,38.829998,20190811T133900Z,20190811T151758.125000Z,-4.485624,0.740238,NaN,NaN,NaN,NaN,NaN
2019-08-11 21:10:59.996822,0.026891,-0.036313,341.965004,2474.961256,1016.163365,1017.380988,7.488089,7.510747,92.048802,91.869156,...,46.950001,20190811T205400Z,20190811T223558.125000Z,-4.368282,-0.470264,NaN,NaN,NaN,NaN,NaN
2019-08-11 22:11:00.000058,0.051789,-0.020034,340.510001,2475.074551,1016.204222,1017.421991,7.507270,7.527613,92.414853,92.252491,...,46.950001,20190811T205400Z,20190811T223558.125000Z,-4.610847,-1.082997,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-21 15:10:59.980900,0.078067,-0.018678,286.894995,2491.681647,996.236889,997.464505,4.346215,4.374028,86.797202,86.444072,...,59.090000,20200121T132400Z,20200121T150558.125000Z,2.753492,10.223569,NaN,NaN,NaN,NaN,NaN
2020-01-21 20:10:59.965464,0.078067,-0.018678,303.655000,2491.649560,994.401416,995.630505,4.236066,4.264247,89.521652,89.357990,...,52.629997,20200121T200600Z,20200121T214758.125000Z,5.230340,7.564519,NaN,NaN,NaN,NaN,NaN
2020-01-21 21:10:59.980005,0.078067,-0.018678,281.330002,2491.649510,994.284652,995.514005,4.237512,4.264786,90.106352,89.978233,...,52.629997,20200121T200600Z,20200121T214758.125000Z,4.118421,7.908356,NaN,NaN,NaN,NaN,NaN


In [14]:
df['t2m'][idx]

nan

In [15]:
air_density(
                    Tair=df['t2m'][idx],
                    Tdew=df['d2m'][idx],
                    MSLP=df['msl'][idx]
                )[0]

nan

In [16]:
for buoy in ascat_dict.keys():
    print(buoy)
    if buoy != 'CLIMODE':
        df = ascat_dict[buoy]
        df['rho_air'] = np.nan
        df['stress_ifs'] = np.nan
        lat = df['lat'][0]
        lon = df['lon'][0]
        lon_360 = lon
        if lon < 0:
            lon_360 = lon + 360
        for idx in df.index:
            if (idx > datetime(2012, 1, 1,)) & (idx < datetime(2017, 1, 1,)): 
            #try:
                # Compute the air density from IFS parameters
                df['rho_air'][idx] = air_density(
                    Tair=df['t2m'][idx],
                    Tdew=df['d2m'][idx],
                    MSLP=df['msl'][idx]/100
                )[0]
                # Compute the stress from IFS parameters
                df['stress_ifs'][idx] = (df['zust'][idx]**2) * df['rho_air'][idx]
            #except:
            #    pass
        # Overwrite df with new columns (IFS)
        ascat_dict[buoy] = df

Endurance_7
Endurance_3
Pioneer_6
Endurance_8
Pioneer_8
Irminger_3
SPURS2
SouthernOcean2
Pioneer_5
Pioneer_3
Pioneer_7
Endurance_4
CLIMODE
Irminger_7
Endurance_9
Pioneer_10
Irminger_6


In [17]:
df['zust'] = np.nan

In [18]:
lat = df['lat'][0]
lon = df['lon'][0]

In [19]:
file_date = str(idx).split(' ')[0].replace('-', '')
ifs_fn = 'ifs_fc_' + '20140503' + '.nc'
ifs_data = xr.open_dataset(ifs_dir + '/' + ifs_fn)

In [20]:
t = '2014-05-03 15:11:00.015259'

In [21]:
ifs_data.sel(
    {
        'time' : t,
        'latitude' : lat,
        'longitude' : lon_360
    },
    method='nearest'
)

<xarray.Dataset>
Dimensions:    ()
Coordinates:
    longitude  float32 320.5
    latitude   float32 59.9
    time       datetime64[ns] 2014-05-03T15:00:00
Data variables:
    chnk       float32 ...
    msl        float32 ...
    t2m        float32 ...
    d2m        float32 ...
    zust       float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-15 04:48:24 GMT by grib_to_netcdf-2.26.0: grib_to_n...

In [22]:
with open(data_dir + 'collocated_dict_ascat_9000_images_ifs_without_spurs1_stress_params.pickle', 'wb') as handle:
    pickle.dump(ascat_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)